In [3]:
from sklearn.model_selection import ParameterGrid
import pandas as pd 
from sqlite3 import connect
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import simplefilter
from scipy.signal import detrend
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set()
import pandasql as psql
import statsmodels.api as sm
import math
import datetime
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 

In [2]:
class model_trainer:
    
    def __init__(self,data):
        self.data=data
    def train(self):
        smoothing=["no","rolling","rolling-gaussian","resample","exponential","exponential2","exponential3"]
        method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        df=self.data.copy()
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
            'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
            'holidays_prior_scale':[0.01,0.1,1,10],
            'seasonality_mode':['additive', 'multiplicative']
            }
        grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "epochs":[100,200,500,1000],
            "batch_size":[8,16,32,64,128],
            'seasonality_mode':('multiplicative','additive'),
            "weekly_seasonality":(True,False),
            "daily_seasonality":(True,False)}
        grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
        for i in smoothing:
            df2=df.copy()
            df2.index=df2["ds"]
            df2.index=pd.to_datetime(df2.index)
            for j in method:
                if i =="no":
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        season=season.fillna(0)
                        mape1=mean_absolute_percentage_error(df["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            model = Prophet(**p)
                            model=model.fit(df)
                            forecast = model.predict(df)
                            mape1=mean_absolute_percentage_error(df["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df, freq="D")
                            forecast = m.predict(df)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="rolling":
                    df2["y"]=df2["y"].rolling(window=7).mean()
                    df2=df2.dropna()
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            model = Prophet(**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="rolling-gaussian":
                    df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
                    df2=df2.dropna()
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            model = Prophet(**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="resample":
                    df2=df2.resample("W").mean()
                    df2=df2.dropna()
                    df2["ds"]=df2.index.tolist()
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            model = Prophet(**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)  
                if i=="exponential":
                    es_simple = SimpleExpSmoothing(df2["y"]).fit()
                    df2["y"]=es_simple.fittedvalues
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season=season.fillna(0)
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            model = Prophet(**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="exponential2":
                    es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= None, seasonal_periods= None).fit()
                    df2["y"]=es_simple.fittedvalues
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            model = Prophet(**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True) 
                if i=="exponential3":
                    es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
                    df2["y"]=es_simple.fittedvalues
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            model = Prophet(**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)

        minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["mape"].argmin()])
        self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
"""             if i =="no":
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        season=season.fillna(0)
                        mape1=mean_absolute_percentage_error(df["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(df)
                            forecast = model.predict(df)
                            mape1=mean_absolute_percentage_error(df["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df, freq="D")
                            forecast = m.predict(df)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)"""

In [ ]:
from sklearn.model_selection import ParameterGrid
import pandas as pd 
from sqlite3 import connect
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import simplefilter
from scipy.signal import detrend
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set()
import pandasql as psql
import statsmodels.api as sm
import math
import datetime
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 



class model_trainer:
    
    def __init__(self,data):
        self.data=data
    def train(self):
        smoothing=["rolling","rolling-gaussian","resample","exponential3"]
        method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        df=self.data.copy()
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "epochs":[100,500,1000],
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
        for i in smoothing:
            df2=df.copy()
            df2.index=df2["ds"]
            df2.index=pd.to_datetime(df2.index)
            for j in method:
                if i=="rolling":
                    df2["y"]=df2["y"].rolling(window=7).mean()
                    df2=df2.dropna()
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="rolling-gaussian":
                    df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
                    df2=df2.dropna()
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="resample":
                    df2=df2.resample("W").mean()
                    df2=df2.dropna()
                    df2["ds"]=df2.index.tolist()
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)  
                if i=="exponential3":
                    es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
                    df2["y"]=es_simple.fittedvalues
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)

        minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["mape"].argmin()])
        self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
from sklearn.model_selection import ParameterGrid
import pandas as pd 
from sqlite3 import connect
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import simplefilter
from scipy.signal import detrend
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set()
import pandasql as psql
import statsmodels.api as sm
import math
import datetime
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit



class model_trainer:
    
    def __init__(self,data):
        self.data=data
    def train(self):
        smoothing=["rolling","rolling-gaussian","resample","exponential3"]
        method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        df=self.data.copy()
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        for i in smoothing:
            df2=df.copy()
            df2.index=df2["ds"]
            df2.index=pd.to_datetime(df2.index)
            for j in method:
                params_grid_neuralprophet = {
                    "learning_rate":[0.01,0.1,1],
                    'seasonality_mode':('multiplicative','additive'),
                    'batch_size':[8,16,32],
                    'weekly_seasonality':(True,False)}
                grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
                if i=="rolling":
                    df2["y"]=df2["y"].rolling(window=7).mean()
                    df2=df2.dropna()
                    tss = TimeSeriesSplit(n_splits = 5)
                    days = np.sort(df2.index.unique())
                    for train_index, test_index in tss.split(df2):
                        train_days, test_days = days[train_index], days[test_index]
                        X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            p["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="rolling-gaussian":
                    df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
                    df2=df2.dropna()
                    tss = TimeSeriesSplit(n_splits = 5)
                    days = np.sort(df2.index.unique())
                    for train_index, test_index in tss.split(df2):
                        train_days, test_days = days[train_index], days[test_index]
                        X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="resample":
                    df2=df2.resample("W").mean()
                    df2=df2.dropna()
                    df2["ds"]=df2.index.tolist()
                    tss = TimeSeriesSplit(n_splits = 5)
                    days = np.sort(df2.index.unique())
                    for train_index, test_index in tss.split(df2):
                        train_days, test_days = days[train_index], days[test_index]
                        X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)  
                if i=="exponential3":
                    es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
                    df2["y"]=es_simple.fittedvalues
                    tss = TimeSeriesSplit(n_splits = 5)
                    days = np.sort(df2.index.unique())
                    for train_index, test_index in tss.split(df2):
                        train_days, test_days = days[train_index], days[test_index]
                        X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index,format="%d-%m-%Y")
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(df2)
                            forecast = model.predict(df2)
                            mape1=mean_absolute_percentage_error(df2["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,**p)
                            m.add_country_holidays(country_name='US')
                            m.fit(df2, freq="D")
                            forecast = m.predict(df2)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)

        minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["mape"].argmin()])
        self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
from sklearn.model_selection import ParameterGrid
import pandas as pd 
from sqlite3 import connect
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import simplefilter
from scipy.signal import detrend
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set()
import pandasql as psql
import statsmodels.api as sm
import math
import datetime
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed



class model_trainer:
    
    def __init__(self,data):
        self.data=data
    def train(self):
        smoothing=["rolling","rolling-gaussian","resample","exponential3"]
        method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        df=self.data.copy()
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        for i in smoothing:
            df2=df.copy()
            df2.index=df2["ds"]
            df2.index=pd.to_datetime(df2.index)
            for j in method:
                params_grid_neuralprophet = {
                    "learning_rate":[0.01,0.1,1],
                    'seasonality_mode':('multiplicative','additive'),
                    'batch_size':[8,16,32],
                    'weekly_seasonality':(True,False)}
                grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
                if i=="rolling":
                    df2["y"]=df2["y"].rolling(window=7).mean()
                    df2=df2.dropna()
                    tss = TimeSeriesSplit(n_splits = 5)
                    days = np.sort(df2.index.unique())
                    for train_index, test_index in tss.split(df2):
                        train_days, test_days = days[train_index], days[test_index]
                        X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            p["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="rolling-gaussian":
                    df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
                    df2=df2.dropna()
                    tss = TimeSeriesSplit(n_splits = 5)
                    days = np.sort(df2.index.unique())
                    for train_index, test_index in tss.split(df2):
                        train_days, test_days = days[train_index], days[test_index]
                        X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            p["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="resample":
                    df2=df2.resample("W").mean()
                    df2=df2.dropna()
                    df2["ds"]=df2.index.tolist()
                    tss = TimeSeriesSplit(n_splits = 5)
                    days = np.sort(df2.index.unique())
                    for train_index, test_index in tss.split(df2):
                        train_days, test_days = days[train_index], days[test_index]
                        X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            p["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                if i=="exponential3":
                    es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
                    df2["y"]=es_simple.fittedvalues
                    tss = TimeSeriesSplit(n_splits = 5)
                    days = np.sort(df2.index.unique())
                    for train_index, test_index in tss.split(df2):
                        train_days, test_days = days[train_index], days[test_index]
                        X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in grid_prophet:
                            print(i,j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in grid_neuralprophet:
                            print(i,j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            p["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':i,'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)

        minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["mape"].argmin()])
        self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
from sklearn.model_selection import ParameterGrid
import pandas as pd 
from sqlite3 import connect
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import simplefilter
from scipy.signal import detrend
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
sns.set()
import pandasql as psql
import statsmodels.api as sm
import math
import datetime
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle


class model_trainer:
    def __init__(self,data):
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def resample(self):
            df2=self.data.copy()
            df2=df2.resample("W").mean()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def minmape(self):
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
            return self.minmape
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle


class model_trainer:
    def __init__(self,data):
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def resample(self):
            df2=self.data.copy()
            df2=df2.resample("W").mean()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle


class model_trainer:
    def __init__(self,data):
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def resample(self):
            df2=self.data.copy()
            df2=df2.resample("W").mean()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle


class model_trainer:
    def __init__(self,data):
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle


class model_trainer:
    def __init__(self,data):
        from pytorch_lightning.callbacks import EarlyStopping
        checkpoint_callback = EarlyStopping(monitor="Loss",mode="min",patience=10)
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters'])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter"},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu"},daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping


class model_trainer:
    def __init__(self,data):
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Taxformsmodels/Taxformsmodel"+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Taxformsmodels/Taxformsmodel"+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Taxformsmodels/Taxformsmodel"+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Taxformsmodels/Taxformsmodel"+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Taxformsmodels/Taxformsmodel"+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Taxformsmodels/Taxformsmodel"+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Taxformsmodels/Taxformsmodel"+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Taxformsmodels/Taxformsmodel"+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping


class model_trainer:
    def __init__(self,data,formsname):
        self.formnanme=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping


class model_trainer:
    def __init__(self,data,formsname):
        self.formnanme=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)],"auto_lr_find":True},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error,accuracy_score
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping


class model_trainer:
    def __init__(self,data,formsname):
        self.formnanme=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[4,8,12,16,32,128],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)],"auto_lr_find":True},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import accuracy_score,accuracy_score
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping


class model_trainer:
    def __init__(self,data,formsname):
        self.formnanme=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[4,8,12,16,32,128],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=accuracy_score(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=accuracy_score(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)],"auto_lr_find":True},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=accuracy_score(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=accuracy_score(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=accuracy_score(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=accuracy_score(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=accuracy_score(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=accuracy_score(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=accuracy_score(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=accuracy_score(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=accuracy_score(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=accuracy_score(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping


class model_trainer:
    def __init__(self,data,formsname):
        self.formnanme=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.05,0.1,0.5,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[4,8,16,32,128],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping


class model_trainer:
    def __init__(self,data,formsname):
        self.formnanme=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping


class model_trainer:
    def __init__(self,data,formsname):
        self.formnanme=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,progress="plot",validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,progress="plot",validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True,progress="plot",validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(model, open(name, 'wb'))
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,progress="plot",validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formnanme+str(self.i)+".pkl"
                            pickle.dump(m, open(name, 'wb'))
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formnanme=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="W-SUN",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=10)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.01,0.1,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.001,0.005,0.01,0.05,0.1,0.5,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(X_train, freq="D",early_stopping=True,checkpointing=True,validation_df=X_test)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet,set_log_level
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.1,0.5,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p,n_changepoints=0)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p,n_changepoints=0)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p,n_changepoints=0)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"resample",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p,n_changepoints=0)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet,set_log_level
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.1,0.5,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p,changepoints=["01-01-2021","01-01-2022","01-01-2023"])
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p,changepoints=["01-01-2021","01-01-2022","01-01-2023"])
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p,changepoints=["01-01-2021","01-01-2022","01-01-2023"])
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p,changepoints=["01-01-2021","01-01-2022","01-01-2023"])
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet,set_log_level
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.1,0.5,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'n_changepoints':[0,1,3,5,8,10],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet,set_log_level
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[0.1,0.5,1],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'n_changepoints':[0,3,5,10],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet,set_log_level
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[1,1.5,2],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'n_changepoints':[0,5,10],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=1000,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet,set_log_level
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["classical","prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[1,1.5,2],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'n_changepoints':[0,5,10],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=100,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=100,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=100,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=100,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_percentage_error
from prophet import Prophet
from neuralprophet import NeuralProphet,set_log_level
import itertools 
from sklearn.model_selection import TimeSeriesSplit
from  neuralprophet import set_random_seed
import pickle
from pytorch_lightning.callbacks import EarlyStopping
import torch

class model_trainer:
    def __init__(self,data,formsname):
        self.formname=formsname
        self.i=0
        self.data=data
        self.method=["prophet","NeuralProphet"]
        self.model_parameters = pd.DataFrame(columns = ['Smoothing','Method','MAPE','Parameters',"Number"])
        params_grid_prophet = {
            'changepoint_prior_scale': [0.001, 0.01, 0.1],
            'seasonality_mode':['additive', 'multiplicative'],
            'weekly_seasonality':(True,False)
            }
        self.grid_prophet= [dict(zip(params_grid_prophet.keys(), v)) for v in itertools.product(*params_grid_prophet.values())]
        params_grid_neuralprophet = {
            "learning_rate":[1,1.5,2],
            'seasonality_mode':('multiplicative','additive'),
            'batch_size':[8,16,32],
            'n_changepoints':[0,3,5,9,10],
            'weekly_seasonality':(True,False)}
        self.grid_neuralprophet= [dict(zip(params_grid_neuralprophet.keys(), v)) for v in itertools.product(*params_grid_neuralprophet.values())]
    def rolling(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7).mean()
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=100,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def rolling_gaussian(self):
            df2=self.data.copy()
            df2["y"]=df2["y"].rolling(window=7,win_type="gaussian").mean(std=10)
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("rolling-gaussian",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("rolling-gaussian",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=100,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"rolling-gaussian",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def Nosmooth(self):
            df2=self.data.copy()
            df2=df2.dropna()
            df2["ds"]=df2.index.tolist()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=52,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("resample",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("resample",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=100,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"Nosmooth",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
    def exponential3(self):
            df2=self.data.copy()
            es_simple = ExponentialSmoothing(df2["y"], trend= 'add', damped= True, seasonal= 'add', seasonal_periods= 365).fit()
            df2["y"]=es_simple.fittedvalues
            df2=df2.dropna()
            tss = TimeSeriesSplit(n_splits = 5)
            days = np.sort(df2.index.unique())
            for train_index, test_index in tss.split(df2):
                train_days, test_days = days[train_index], days[test_index]
                X_train, X_test = df2.loc[train_days,], df2.loc[test_days,]
            for j in self.method:
                    if j=="classical":
                        decomposeresult2=seasonal_decompose(df2["y"],period=365,model="add")
                        season = pd.DataFrame({"obs":decomposeresult2.observed,"trend":decomposeresult2.trend,"seasonal":decomposeresult2.seasonal})
                        season.index=pd.to_datetime(season.index)
                        season=season.fillna(0)
                        season["diff"]=season["obs"].sub(season["trend"],axis=0)
                        mape1=mean_absolute_percentage_error(df2["y"], (season.seasonal+season.trend))
                        self.i+=1
                        self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':"No parameter","Number":self.i},ignore_index=True)
                    if j=="prophet":
                        for p in self.grid_prophet:
                            print("exponential3",j,p)
                            set_log_level("ERROR")
                            model = Prophet(daily_seasonality=False,**p)
                            model=model.fit(X_train)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(model, name)
                            forecast = model.predict(X_test)
                            mape1=mean_absolute_percentage_error(X_test["y"], forecast["yhat"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':p,"Number":self.i},ignore_index=True)
                    if j=="NeuralProphet":
                        for p in self.grid_neuralprophet:
                            print("exponential3",j,p)
                            set_random_seed(0)
                            set_log_level("ERROR")
                            m = NeuralProphet(trainer_config={"accelerator":"gpu","callbacks":[EarlyStopping(monitor="Loss_val",mode="min",patience=5)]},
                                              accelerator="gpu",daily_seasonality=False,epochs=100,**p)
                            df_train, df_val = m.split_df(df=X_train, freq="D", valid_p=0.1)
                            m.add_country_holidays(country_name='US')
                            df_model=m.fit(df_train, freq="D",early_stopping=True,validation_df=df_val)
                            self.i+=1
                            name="/home/batuhan-saylam/Desktop/JotformProject/projects/Formmodels/"+self.formname+str(self.i)+".pt"
                            torch.save(m, name)
                            k=p.copy()
                            k["epoch" ]=df_model.iloc[-1,4]
                            forecast = m.predict(X_test)
                            mape1=mean_absolute_percentage_error(forecast["y"], forecast["yhat1"])
                            self.model_parameters = self.model_parameters.append({'Smoothing':"exponential3",'Method':j,'MAPE':mape1,'Parameters':k,"Number":self.i},ignore_index=True)
            minmape=pd.DataFrame(self.model_parameters.loc[self.model_parameters["MAPE"].argmin()])
            self.minmape=minmape.T
    def picking(self,filename):
        with open(filename, 'wb') as file:  
            pickle.dump(self.model_parameters, file)
    